<a href="https://colab.research.google.com/github/soniamaryc/ARUI_RandomObjectManipulation/blob/master/IK_Jacobian_JointVelocities_2LinkPlanar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# Robot Parameters
L1, L2 = 1.0, 1.0  # Link lengths

# Jacobian Matrix Calculation
def jacobian(theta):
    theta1, theta2 = theta
    J = np.array([
        [-L1 * np.sin(theta1) - L2 * np.sin(theta1 + theta2), -L2 * np.sin(theta1 + theta2)],
        [ L1 * np.cos(theta1) + L2 * np.cos(theta1 + theta2),  L2 * np.cos(theta1 + theta2)]
    ])
    return J

# Function to calculate joint velocities given end-effector velocities
def calculate_joint_velocities(theta, end_effector_vel):
    J = jacobian(theta)
    J_inv = np.linalg.pinv(J)
    joint_velocities = J_inv @ end_effector_vel
    return joint_velocities

# Example Usage
if __name__ == "__main__":
    # Current joint angles (in radians)
    theta = np.array([np.pi/4, np.pi/4])

    # Desired end-effector velocity (x_dot, y_dot)
    end_effector_vel = np.array([0.1, 0.0])  # Example: move horizontally at 0.1 m/s

    # Compute joint velocities
    joint_velocities = calculate_joint_velocities(theta, end_effector_vel)

    # Display results
    print("Joint Angles (rad):", theta)
    print("Desired End-Effector Velocity (m/s):", end_effector_vel)
    print("Calculated Joint Velocities (rad/s):", joint_velocities)


Joint Angles (rad): [0.78539816 0.78539816]
Desired End-Effector Velocity (m/s): [0.1 0. ]
Calculated Joint Velocities (rad/s): [ 1.92490079e-17 -1.00000000e-01]


In [ ]:
import numpy as np

# Robot Parameters
L1, L2 = 1.0, 1.0  # Link lengths in meters

# Forward Kinematics Function (for reference, not directly used here)
def forward_kinematics(theta1, theta2):
    x = L1 * np.cos(theta1) + L2 * np.cos(theta1 + theta2)
    y = L1 * np.sin(theta1) + L2 * np.sin(theta1 + theta2)
    return np.array([x, y])

# Jacobian Matrix Calculation
def compute_jacobian(theta1, theta2):
    """
    Compute the 2x2 Jacobian for a 2-link planar robot.
    theta1, theta2: joint angles in radians
    Returns: Jacobian matrix J
    """
    J11 = -L1 * np.sin(theta1) - L2 * np.sin(theta1 + theta2)  # ∂x/∂θ₁
    J12 = -L2 * np.sin(theta1 + theta2)                        # ∂x/∂θ₂
    J21 = L1 * np.cos(theta1) + L2 * np.cos(theta1 + theta2)   # ∂y/∂θ₁
    J22 = L2 * np.cos(theta1 + theta2)                         # ∂y/∂θ₂
    return np.array([[J11, J12], [J21, J22]])

# Joint Velocity Solver
def compute_joint_velocities(desired_velocity, theta1, theta2):
    """
    Calculate joint velocities θ̇ given desired end-effector velocity ẋ.
    desired_velocity: [ẋ, ẏ] in m/s
    theta1, theta2: current joint angles in radians
    Returns: [θ̇₁, θ̇₂] in rad/s
    """
    # Convert desired velocity to numpy array
    v = np.array(desired_velocity, dtype=float).reshape(2, 1)  # Column vector

    # Compute Jacobian at current configuration
    J = compute_jacobian(theta1, theta2)

    # Check for singularity
    det_J = np.linalg.det(J)
    if abs(det_J) < 1e-6:
        print("Warning: Jacobian is singular. Using pseudoinverse instead.")
        J_inv = np.linalg.pinv(J)  # Fallback to pseudoinverse
    else:
        J_inv = np.linalg.inv(J)   # Direct inverse if invertible

    # Solve for joint velocities: θ̇ = J⁻¹ * ẋ
    theta_dot = J_inv @ v
    return theta_dot.flatten()  # Return as 1D array [θ̇₁, θ̇₂]

# Test the function
def main():
    # Current joint angles (example configuration)
    theta1 = 0.0           # 0° (base horizontal)
    theta2 = np.pi / 2     # 90° (second link vertical)

    # Desired end-effector velocity (m/s)
    desired_velocity = [0.5, 0.5]  # ẋ = 0.5 m/s, ẏ = 0.5 m/s

    # Calculate joint velocities
    theta_dot = compute_joint_velocities(desired_velocity, theta1, theta2)

    # Print results
    print(f"Current Configuration: θ₁ = {np.degrees(theta1):.2f}°, θ₂ = {np.degrees(theta2):.2f}°")
    print(f"Desired End-Effector Velocity: ẋ = {desired_velocity[0]} m/s, ẏ = {desired_velocity[1]} m/s")
    print(f"Joint Velocities: θ̇₁ = {theta_dot[0]:.3f} rad/s, θ̇₂ = {theta_dot[1]:.3f} rad/s")

    # Verify: Compute resulting end-effector velocity
    J = compute_jacobian(theta1, theta2)
    v_computed = J @ theta_dot.reshape(2, 1)
    print(f"Computed End-Effector Velocity: ẋ = {v_computed[0, 0]:.3f} m/s, ẏ = {v_computed[1, 0]:.3f} m/s")

if __name__ == "__main__":
    main()

Current Configuration: θ₁ = 0.00°, θ₂ = 90.00°
Desired End-Effector Velocity: ẋ = 0.5 m/s, ẏ = 0.5 m/s
Joint Velocities: θ̇₁ = 0.500 rad/s, θ̇₂ = -1.000 rad/s
Computed End-Effector Velocity: ẋ = 0.500 m/s, ẏ = 0.500 m/s


**Minimalist Code**

In [ ]:
import numpy as np

# Robot link lengths
L1, L2 = 1.0, 1.0

# Joint angles (in radians)
theta1, theta2 = 0.0, np.pi / 2  # 0° and 90°

# Desired end-effector velocity (m/s)
vx, vy = 0.5, 0.5

# Jacobian matrix (2x2)
J = np.array([
    [-L1 * np.sin(theta1) - L2 * np.sin(theta1 + theta2), -L2 * np.sin(theta1 + theta2)],
    [L1 * np.cos(theta1) + L2 * np.cos(theta1 + theta2), L2 * np.cos(theta1 + theta2)]
])

# Inverse Jacobian and joint velocities
J_inv = np.linalg.inv(J)              # Invert the Jacobian
v = np.array([vx, vy])                # Velocity vector
theta_dot = J_inv @ v                 # θ̇ = J⁻¹ * v

# Print results
print(f"θ̇₁ = {theta_dot[0]:.3f} rad/s, θ̇₂ = {theta_dot[1]:.3f} rad/s")

θ̇₁ = 0.500 rad/s, θ̇₂ = -1.000 rad/s
